## Convert Fig8 Annotations to LabelMe Format
This notebook allows one to import and edit if necessary the crowdsourced labels from `figure-eight.com` locally in the PyQt-based [LabelMe](https://github.com/wkentaro/labelme) tool.

In [ ]:
import os
import json

In [ ]:
# note, first have to wrap the json annotations in '{}' and 
# comma-separate entries

job_path = '/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Train/2017-08/'
job_file = 'job_1538711.json'
filename = os.path.join(job_path, job_file)

with open(filename, 'r') as f:
    att = json.load(f)
print('Found labels for %d images' % len(att['foo']))

In [ ]:
def create_labelme_annotation_dict(jpeg_filename, points_list):
    """create a new python dict in LabelMe format for
    conversion to json object
    @param points_list list of vertices
    """
    data = {}
    data['version'] = '3.21.1'
    data['flags'] = {}
    data['shapes'] = []
    for l in range(len(points_list)):
        data['shapes'].append({
            'label': "zebra",
            'line_color': None,
            'fill_color': None,
            'points': points_list[l],
            'shape_type': "polygon",
            'flags' : {}
        })
    data['lineColor'] = (0, 255, 0, 128)
    data['fillColor'] = (255, 0, 0, 128)
    data['imagePath'] = jpeg_filename
    data['imageData'] = None
    data['imageHeight'] = 2250
    data['imageWidth'] = 2250
    
    return data

In [ ]:
#img_idx = 19
#att['foo'][img_idx]['results']['judgments'][2]['data']['annotation']

In [ ]:
#att['foo'][0]['data']['image_url'].split('/')[-1]

In [ ]:
# loop over all images that have at least one judgment
for img_idx in range(len(att['foo'])):
    jpeg_filename = att['foo'][img_idx]['data']['image_url'].split('/')[-1]

    # loop over all worker judgments
    points_list = [] # initialize list that will contain all vertices for a given image
    for worker in range(len(att['foo'][img_idx]['results']['judgments'])):
        #print('worker ', worker)
        att_json = json.loads(att['foo'][img_idx]['results']['judgments'][worker]['data']['annotation'])

        # for number of distinct polygons in annotation
        for j in range(len(att_json)):
            vertices_list = []
            # for number of vertices in each polygon
            for k in range(len(att_json[j]['coordinates'])):
                #print(att_json[i]['coordinates'][j])
                vertices_list.append(
                    [att_json[j]['coordinates'][k]['x'],
                     att_json[j]['coordinates'][k]['y']])
            points_list.append(vertices_list)

# get a dictionary containing polygon vertices in labelme format
data = create_labelme_annotation_dict(jpeg_filename, points_list)

# save the json file with the same root as the associated image
with open(os.path.join(job_path, jpeg_filename.split('.')[0] + '.json'), 'w') as outfile:
    json.dump(data, outfile, indent=2)